In [1]:
#Problem Statement
# It happens all the time: someone gives you data containing malformed strings,
# Python, lists and missing data. How do you tidy it up so you can get on with the
# analysis?
# Take this monstrosity as the DataFrame to use in the following puzzles:
import pandas as pd
import numpy as np
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN','londON_StockhOlm','Budapest_PaRis', 'Brussels_londOn'],
                    'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],
                    'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]],
                    'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )','12. Air France', '"Swiss Air"']})

In [2]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


In [3]:
#1. Some values in the the FlightNumber column are missing. 
# These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. 
# Fill in these missing numbers and make the column an integer column (instead of a float column).
print(df.FlightNumber.isnull())
def enumerate_fill(x):
    for i,j in enumerate(x):
        if i==0:
            x[i] = x[i]
        else:
            x[i] = x[i-1]+10
enumerate_fill(df['FlightNumber'])
df['FlightNumber'] = df['FlightNumber'].astype(int)

0    False
1     True
2    False
3     True
4    False
Name: FlightNumber, dtype: bool


C:\Users\rames\AppData\Local\Temp/ipykernel_2820/3411268529.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = x[i]
C:\Users\rames\AppData\Local\Temp/ipykernel_2820/3411268529.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i] = x[i-1]+10


In [4]:
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


In [5]:
#2. The From_To column would be better as two separate columns! 
# Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. 
# Assign the correct column names to this temporary DataFrame.
df1 = pd.DataFrame(df, columns=['From_To'])

In [6]:
df1['From'] = df['From_To'].apply(lambda x: x.split('_')[0])
df1

,From_To,From
0,LoNDon_paris,LoNDon
1,MAdrid_miLAN,MAdrid
2,londON_StockhOlm,londON
3,Budapest_PaRis,Budapest
4,Brussels_londOn,Brussels


In [7]:
df1['To'] = df['From_To'].apply(lambda x: x.split('_')[1])
df1

,From_To,From,To
0,LoNDon_paris,LoNDon,paris
1,MAdrid_miLAN,MAdrid,miLAN
2,londON_StockhOlm,londON,StockhOlm
3,Budapest_PaRis,Budapest,PaRis
4,Brussels_londOn,Brussels,londOn


In [8]:
df1.drop('From_To',axis=1,inplace=True)
df1

,From,To
0,LoNDon,paris
1,MAdrid,miLAN
2,londON,StockhOlm
3,Budapest,PaRis
4,Brussels,londOn


In [9]:
#3. Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. 
# Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".)
df1['From'] = df1['From'].apply(lambda x: x.title())
df1['To'] = df1['To'].apply(lambda x: x.title())

In [10]:
df1

,From,To
0,London,Paris
1,Madrid,Milan
2,London,Stockholm
3,Budapest,Paris
4,Brussels,London


In [11]:
#4. Delete the From_To column from df and attach the temporary DataFrame from the previous questions.
df.drop('From_To', axis=1, inplace=True)

In [12]:
df = pd.concat([df, df1], axis=1)

In [13]:
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045,"[23, 47]",KLM(!),London,Paris
1,10055,[],<Air France> (12),Madrid,Milan
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075,[13],12. Air France,Budapest,Paris
4,10085,"[67, 32]","""Swiss Air""",Brussels,London


In [14]:
#5. In the RecentDelays column, the values have been entered into the DataFrame as a list. 
# We would like each first value in its own column, each second value in its own column, and so on. 
# If there isn't an Nth value, the value should be NaN.

# Expand the Series of lists into a DataFrame named delays, 
# rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays.

In [15]:
# Expand the Series of lists into a DataFrame named delays
delays = pd.DataFrame(df['RecentDelays'].values.tolist())

In [16]:
delays

,0,1,2
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [17]:
delays.dtypes

0    float64
1    float64
2    float64
dtype: object

In [18]:
column_dict = {}
for i in range(delays.shape[1]):
    key = delays.columns[i]
    value = 'delays_'+str(i+1)
    column_dict[key] = value
column_dict

{0: 'delays_1', 1: 'delays_2', 2: 'delays_3'}

In [19]:
# rename the columns delay_1, delay_2, etc.
delays.rename(columns=column_dict, inplace=True)

In [20]:
delays

,delays_1,delays_2,delays_3
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [21]:
df

,FlightNumber,RecentDelays,Airline,From,To
0,10045,"[23, 47]",KLM(!),London,Paris
1,10055,[],<Air France> (12),Madrid,Milan
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm
3,10075,[13],12. Air France,Budapest,Paris
4,10085,"[67, 32]","""Swiss Air""",Brussels,London


In [22]:
df = pd.concat([df, delays], axis=1)

In [23]:
df

,FlightNumber,RecentDelays,Airline,From,To,delays_1,delays_2,delays_3
0,10045,"[23, 47]",KLM(!),London,Paris,23.0,47.0,NaN
1,10055,[],<Air France> (12),Madrid,Milan,NaN,NaN,NaN
2,10065,"[24, 43, 87]",(British Airways. ),London,Stockholm,24.0,43.0,87.0
3,10075,[13],12. Air France,Budapest,Paris,13.0,NaN,NaN
4,10085,"[67, 32]","""Swiss Air""",Brussels,London,67.0,32.0,NaN


In [24]:
#replace the unwanted RecentDelays column in df with delays.
df.drop('RecentDelays', axis=1, inplace=True)

In [25]:
#Final Dataframe
df

,FlightNumber,Airline,From,To,delays_1,delays_2,delays_3
0,10045,KLM(!),London,Paris,23.0,47.0,NaN
1,10055,<Air France> (12),Madrid,Milan,NaN,NaN,NaN
2,10065,(British Airways. ),London,Stockholm,24.0,43.0,87.0
3,10075,12. Air France,Budapest,Paris,13.0,NaN,NaN
4,10085,"""Swiss Air""",Brussels,London,67.0,32.0,NaN
